# Deep Feature Synthesis

Deep Feature Synthesis (DFS) is an automated method for performing feature engineering on relational and temporal data.

## Input Data

Deep Feature Synthesis requires structured datasets in order to perform feature engineering. To demonstrate the capabilities of DFS, we will use a mock customer transactions dataset.


In [ ]:
import featuretools as ft

es = ft.demo.load_mock_customer(return_entityset=True)
es

Once data is prepared as an `.EntitySet`, we are ready to automatically generate features for a target dataframe - e.g. `customers`.

## Running DFS

Typically, without automated feature engineering, a data scientist would write code to aggregate data for a customer, and apply different statistical functions resulting in features quantifying the customer's behavior. In this example, an expert might be interested in features such as: *total number of sessions* or *month the customer signed up*.

These features can be generated by DFS when we specify the target_dataframe as `customers` and `"count"` and `"month"` as primitives.

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    agg_primitives=["count"],
    trans_primitives=["month"],
    max_depth=1,
)
feature_matrix

In the example above, `"count"` is an **aggregation primitive** because it computes a single value based on many sessions related to one customer. `"month"` is called a **transform primitive** because it takes one value for a customer transforms it to another.

## Creating "Deep Features"

The name Deep Feature Synthesis comes from the algorithm's ability to stack primitives to generate more complex features. Each time we stack a primitive we increase the "depth" of a feature. The `max_depth` parameter controls the maximum depth of the features returned by DFS. Let us try running DFS with `max_depth=2`

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    agg_primitives=["mean", "sum", "mode"],
    trans_primitives=["month", "hour"],
    max_depth=2,
)
feature_matrix

With a depth of 2, a number of features are generated using the supplied primitives. The algorithm to synthesize these definitions is described in this [paper](https://www.jmaxkanter.com/papers/DSAA_DSM_2015.pdf). In the returned feature matrix, let us understand one of the depth 2 features

In [ ]:
feature_matrix[["MEAN(sessions.SUM(transactions.amount))"]]

For each customer this feature

1. calculates the ``sum`` of all transaction amounts per session to get total amount per session,
2. then applies the ``mean`` to the total amounts across multiple sessions to identify the *average amount spent per session*

We call this feature a "deep feature" with a depth of 2.

Let's look at another depth 2 feature that calculates for every customer *the most common hour of the day when they start a session*

In [ ]:
feature_matrix[["MODE(sessions.HOUR(session_start))"]]

For each customer this feature calculates

1. The `hour` of the day each of his or her sessions started, then
2. uses the statistical function `mode` to identify the most common hour he or she started a session

Stacking results in features that are more expressive than individual primitives themselves. This enables the automatic creation of complex patterns for machine learning.

## Changing Target DataFrame

DFS is powerful because we can create a feature matrix for any dataframe in our dataset. If we switch our target dataframe to "sessions", we can synthesize features for each session instead of each customer. Now, we can use these features to predict the outcome of a session.

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="sessions",
    agg_primitives=["mean", "sum", "mode"],
    trans_primitives=["month", "hour"],
    max_depth=2,
)
feature_matrix.head(5)

As we can see, DFS will also build deep features based on a parent dataframe, in this case the customer of a particular session. For example, the feature below calculates the mean transaction amount of the customer of the session.

In [ ]:
feature_matrix[["customers.MEAN(transactions.amount)"]].head(5)